In [1]:
import numpy as np
import pandas as pd
import re
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import time
import os

import neuro_morpho_toolbox as nmt
from neuro_morpho_toolbox.flat_map import *

import sklearn
from sklearn.decomposition import PCA
import umap
from sklearn import metrics

import pickle
%matplotlib inline


/Users/pengxie/Documents/Research/python/neuro_morhpo_toolbox/neuro_morpho_toolbox/
Loading CCF Atlas data...
Loading time: 0.54
Loading CCF brain structure data...
Loading time: 0.00
Loading flat_map ...
Loading time: 2.27


In [2]:
def sum_regions(x, region='SSp'):
    for tag in ['-ipsi', '-contra']:
        tp_list = [nmt.bs.id_to_name(i)+tag for i in nmt.bs.get_all_child_id(region)]
        tp_list = [i for i in tp_list if i in (x.columns.tolist())]
        res = x[tp_list].sum(axis=1)
        x.drop(columns=tp_list, inplace=True)
        x[region+tag] = res
    return x



In [3]:
[ns] = pickle.load(open('./neuron_set_1741cells.pickle', 'rb'))

In [4]:
pdf = ns.features['projection_features'].raw_data.copy() / 1000
pdf.drop(columns=['ipsi_fiber tracts', 'contra_fiber tracts'], inplace=True)

old_col = pdf.columns.tolist()
new_col = []
for i in old_col:
    if 'ipsi' in i:
        i = i.replace('ipsi_', '')+'-ipsi'
    else:
        i = i.replace('contra_', '')+'-contra'
    new_col.append(i)
pdf.rename(columns=dict(zip(old_col, new_col)), inplace=True)
pdf = sum_regions(pdf, 'SSp')


In [5]:
metadata = ns.metadata.copy()
metadata['Transgenic_line'] = [str(i).split('-')[0] for i in metadata['Transgenic_line'].tolist()]
metadata.loc[metadata['Manually_corrected_soma_region'].str.startswith('SSp'), 
             'Manually_corrected_soma_region']='SSp'

metadata['N_targets'] = (pdf>1).sum(axis=1).loc[metadata.index].tolist()

metadata['Manually_corrected_soma_region'].value_counts()

VPM      378
CP       312
SSp      233
VPL       80
LGd       78
        ... 
PB         1
 RSPv      1
ADUv       1
SGN        1
FRP        1
Name: Manually_corrected_soma_region, Length: 82, dtype: int64

In [6]:
metadata['Subclass_or_type'].value_counts()

TH_core      637
CTX_ET       238
CTX_IT       217
CP_GPe       180
Car3         102
CP_SNr        96
TH_matrix     63
CP_others     31
Name: Subclass_or_type, dtype: int64

In [7]:
def get_clist(Transgenic_line, Manually_corrected_soma_region=None, Cortical_layer=None, Subclass_or_type=None):
    tp = metadata[metadata['Transgenic_line'].isin(Transgenic_line)].copy()
    if Manually_corrected_soma_region is not None:
        tp = tp[tp['Manually_corrected_soma_region'].isin(Manually_corrected_soma_region)]
    if Cortical_layer is not None:
        tp = tp[tp['Cortical_layer'].isin(Cortical_layer)]
    if Subclass_or_type is not None:
        tp = tp[tp['Subclass_or_type'].isin(Subclass_or_type)]
    clist = tp.index.tolist()
    return clist

clist = get_clist(Transgenic_line=['Cux2'])
metadata.loc[clist]['Subclass_or_type'].value_counts()

CTX_IT    162
Name: Subclass_or_type, dtype: int64

In [27]:
metadata['grouping'] = 'others'

tag = 'Cux2_L2/3_IT'
clist = get_clist(Transgenic_line=['Cux2'],
                  Manually_corrected_soma_region=['MOp', 'MOs', 'SSp', 'SSs'],
                  Subclass_or_type=['CTX_IT']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag
print(metadata.loc[clist,'Manually_corrected_soma_region'].value_counts())

Number of cells:	101
SSp    46
MOp    22
MOs    18
SSs    15
Name: Manually_corrected_soma_region, dtype: int64


In [8]:
metadata['grouping'] = 'others'

tag = 'Cux2_L2/3_IT'
clist = get_clist(Transgenic_line=['Cux2'],
                  Manually_corrected_soma_region=['MOp', 'MOs', 'SSp', 'SSs'],
#                   Manually_corrected_soma_region=['SSp'],
                  Cortical_layer=['2/3'],
                  Subclass_or_type=['CTX_IT']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	67


In [9]:
metadata['grouping'] = 'others'

tag = 'Cux2_L2/3_IT'
clist = get_clist(Transgenic_line=['Cux2'],
                  Manually_corrected_soma_region=['MOp', 'MOs', 'SSp', 'SSs'],
                  Cortical_layer=['4'],
                  Subclass_or_type=['CTX_IT']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag
print(metadata.loc[clist,'Manually_corrected_soma_region'].value_counts())

Number of cells:	33
SSp    26
MOp     3
SSs     3
MOs     1
Name: Manually_corrected_soma_region, dtype: int64


In [11]:
tag = 'Plxnd1_L5_IT'
clist = get_clist(Transgenic_line=['Plxnd1'],
                  Manually_corrected_soma_region=['MOp', 'MOs', 'SSp', 'SSs'],
#                   Manually_corrected_soma_region=['SSp'],
                  Cortical_layer=['5'],
                  Subclass_or_type=['CTX_IT']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	33


In [12]:
tag = 'Gnb4_L6_Car3'
clist = get_clist(Transgenic_line=['Gnb4'],
                  Cortical_layer=['6'],
                  Subclass_or_type=['Car3']
                 )
clist = metadata.loc[((metadata['Manually_corrected_soma_region']!='CLA') & (metadata.index.isin(clist)))].index.tolist()
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	71


In [13]:
tag = 'Gnb4_CLA'
clist = get_clist(Transgenic_line=['Gnb4'],
                  Manually_corrected_soma_region=['CLA'],
                  Subclass_or_type=['Car3']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	29


In [14]:
MY_regions = [nmt.bs.id_to_name(i) for i in nmt.bs.get_all_child_id('MY') if i in nmt.bs.selected_regions]
MY_regions = sorted(MY_regions)

my_list = [i+'-ipsi' for i in MY_regions] + [i+'-contra' for i in MY_regions]
my_list = [i for i in my_list if i in pdf.columns.tolist()]

def whether_medula(clist):
    tp = pdf.loc[clist]
    tp['MY'] = tp[my_list].sum(axis=1)
    tp.drop(columns=my_list, inplace=True)
    res = tp['MY']>1
    return res

clist = get_clist(Transgenic_line=['Fezf2', 'Pvalb'],
                  Manually_corrected_soma_region=['MOs', 'MOp', 'SSp', 'SSs'],
                  Cortical_layer=['5'],
                  Subclass_or_type=['CTX_ET']
                 )
print('Number of cells:\t%d' % (len(clist)))

tp = metadata.loc[clist].copy()
tp['Medulla'] = whether_medula(clist).tolist()

tag = 'Fezf2/Pvalb_L5_ET_MY'
clist1 = tp[((tp['Manually_corrected_soma_region'].isin(['MOs', 'MOp', 'SSp'])) & (tp['Medulla']))].index.tolist()
print('Number of cells:\t%d' % (len(clist1)))
metadata.loc[clist1, 'grouping'] = tag

tag = 'Fezf2/Pvalb_L5_ET_nonMY'
clist2 = tp[((tp['Manually_corrected_soma_region'].isin(['MOs', 'MOp', 'SSp', 'SSs'])) & (~tp['Medulla']))].index.tolist()
print('Number of cells:\t%d' % (len(clist2)))
metadata.loc[clist2, 'grouping'] = tag


Number of cells:	196
Number of cells:	45
Number of cells:	147


In [15]:
tag = 'Tnnt1/Vipr2_TH_core'
clist = get_clist(Transgenic_line=['Tnnt1', 'Vipr2'],
                  Subclass_or_type=['TH_core']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	637


In [16]:
tag = 'Tnnt1/Vipr2_TH_matrix'
clist = get_clist(Transgenic_line=['Tnnt1', 'Vipr2'],
                  Subclass_or_type=['TH_matrix']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	63


In [26]:
th_list = [nmt.bs.id_to_name(i) for i in nmt.bs.get_all_child_id('TH')]
clist = get_clist(Transgenic_line=['Tnnt1', 'Vipr2', 'Pvalb'],
                  Manually_corrected_soma_region=th_list
                 )
print('Number of cells:\t%d' % (len(clist)))
print(len(metadata.loc[clist, 'Manually_corrected_soma_region'].unique()))

Number of cells:	735
21


In [17]:
tag = 'Tnnt1/Vipr2/Plxnd1_CP_SNr'
clist = get_clist(Transgenic_line=['Tnnt1', 'Vipr2', 'Plxnd1'],
                  Subclass_or_type=['CP_SNr']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	94


In [18]:
tag = 'Tnnt1/Vipr2/Plxnd1_CP_GPe'
clist = get_clist(Transgenic_line=['Tnnt1', 'Vipr2', 'Plxnd1'],
                  Subclass_or_type=['CP_GPe']
                 )
print('Number of cells:\t%d' % (len(clist)))
metadata.loc[clist, 'grouping'] = tag

Number of cells:	179


In [19]:
metadata.groupby('grouping')['N_targets'].mean()

grouping
Cux2_L4_IT                    2.393939
Fezf2/Pvalb_L5_ET_MY         16.155556
Fezf2/Pvalb_L5_ET_nonMY       8.863946
Gnb4_CLA                     16.379310
Gnb4_L6_Car3                 10.084507
Plxnd1_L5_IT                 11.303030
Tnnt1/Vipr2/Plxnd1_CP_GPe     1.960894
Tnnt1/Vipr2/Plxnd1_CP_SNr     2.978723
Tnnt1/Vipr2_TH_core           2.854003
Tnnt1/Vipr2_TH_matrix         7.238095
others                        6.063415
Name: N_targets, dtype: float64

In [20]:
metadata[metadata['grouping']=='Plxnd1_L5_IT'].sort_values('N_targets', ascending=False)

,formatted name,Soma_x,Soma_y,Soma_z,Registered_soma_region,Manually_corrected_soma_region,Cortical_layer,Transgenic_line,Brain_id,Subclass_or_type,Notes,N_targets,grouping
191812_6084_x5026_y7787,NaN,7198.075,1282.725,2791.575,SSp-bfd,SSp,5,Plxnd1,191812,CTX_IT,NaN,20,Plxnd1_L5_IT
191812_3758_x8791_y24162,NaN,4587.525,3922.975,2049.725,SSs,SSs,5,Plxnd1,191812,CTX_IT,NaN,19,Plxnd1_L5_IT
191812_5586_x4875_y23378,NaN,6673.800,1758.825,2418.750,SSp-bfd,SSp,5,Plxnd1,191812,CTX_IT,NaN,18,Plxnd1_L5_IT
191812_4775_x5212_y22490,NaN,5755.850,1830.425,2931.800,SSp-un,SSp,5,Plxnd1,191812,CTX_IT,NaN,18,Plxnd1_L5_IT
191812_5221_x9028_y25900,NaN,6125.125,3925.800,1375.950,SSs,SSs,5,Plxnd1,191812,CTX_IT,NaN,17,Plxnd1_L5_IT
191812_3915_x8011_y24180,NaN,4819.100,3519.250,2040.425,SSp-m,SSp,5,Plxnd1,191812,CTX_IT,NaN,17,Plxnd1_L5_IT
17787_4521_x5172_y12873,17787_4521_x5172_y12873,5437.125,4442.275,1227.250,VISC,SSs,5,Plxnd1,17787,CTX_IT,NaN,15,Plxnd1_L5_IT
191812_5922_x4572_y8645,NaN,6952.500,1094.050,3130.975,SSp-tr,SSp,5,Plxnd1,191812,CTX_IT,NaN,15,Plxnd1_L5_IT
191812_4664_x8027_y25176,191812_4664_x8027_y25176,5810.850,3191.775,1629.250,SSs,SSs,5,Plxnd1,191812,CTX_IT,NaN,15,Plxnd1_L5_IT
191812_3922_x8455_y6768,NaN,4846.950,3429.325,1924.425,SSp-m,SSp,5,Plxnd1,191812,CTX_IT,NaN,14,Plxnd1_L5_IT


In [21]:
tp = pdf.loc['191812_6084_x5026_y7787']
tp.sort_values(ascending=False).head(20)

SSp-ipsi       45.690002
ECT-ipsi       10.249208
ENTl-contra     9.456177
ECT-contra      6.022704
CP-ipsi         5.576278
VISrl-ipsi      5.150782
TEa-ipsi        5.115620
TEa-contra      3.625676
SSp-contra      3.308306
VISa-ipsi       3.252127
VISpor-ipsi     2.780736
PERI-contra     2.049067
SSs-contra      1.828736
VISl-ipsi       1.821227
SSs-ipsi        1.583316
AUDv-contra     1.468386
VISC-ipsi       1.449110
VISal-ipsi      1.125259
VISli-ipsi      1.064880
PIR-contra      1.018106
Name: 191812_6084_x5026_y7787, dtype: float64